In [1]:
import pandas as pd
df_news = pd.read_json("News_Category_Dataset_v2.json", lines = True)
df_news.columns

Index(['category', 'headline', 'authors', 'link', 'short_description', 'date'], dtype='object')

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string

stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

my_sw = ['make', 'amp',  'news','new' ,'time', 'u','s', 'photos',  'get', 'say']
def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2 and token not in my_sw
  
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text)    
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [3]:
df_news.category = df_news.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

In [4]:
df_news['text'] = df_news['headline'] + " " + df_news['short_description']

In [8]:
import re
df_news['clean_text']=df_news['text'].apply(clean_txt)

In [10]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

df_news['tokens']=df_news['clean_text'].apply(tokenizer.tokenize)

In [11]:
df_news

category                                           headline  \
0               CRIME  There Were 2 Mass Shootings In Texas Last Week...   
1       ENTERTAINMENT  Will Smith Joins Diplo And Nicky Jam For The 2...   
2       ENTERTAINMENT    Hugh Grant Marries For The First Time At Age 57   
3       ENTERTAINMENT  Jim Carrey Blasts 'Castrato' Adam Schiff And D...   
4       ENTERTAINMENT  Julianna Margulies Uses Donald Trump Poop Bags...   
...               ...                                                ...   
200848           TECH  RIM CEO Thorsten Heins' 'Significant' Plans Fo...   
200849         SPORTS  Maria Sharapova Stunned By Victoria Azarenka I...   
200850         SPORTS  Giants Over Patriots, Jets Over Colts Among  M...   
200851         SPORTS  Aldon Smith Arrested: 49ers Linebacker Busted ...   
200852         SPORTS  Dwight Howard Rips Teammates After Magic Loss ...   

                 authors                                               link  \
0        Melissa Jeltsen  https://www.huffingtonpost.com/entry/texas-ama...   
1          Andy McDonald  https://www.huffingtonpost.com/entry/will-smit...   
2             Ron Dicker  https://www.huffingtonpost.com/entry/hugh-gran...   
3             Ron Dicker  https://www.huffingtonpost.com/entry/jim-carre...   
4             Ron Dicker  https://www.huffingtonpost.com/entry/julianna-...   
...                  ...                                                ...   
200848  Reuters, Reuters  https://www.huffingtonpost.com/entry/rim-ceo-t...   
200849                    https://www.huffingtonpost.com/entry/maria-sha...   
200850                    https://www.huffingtonpost.com/entry/super-bow...   
200851                    https://www.huffingtonpost.com/entry/aldon-smi...   
200852                    https://www.huffingtonpost.com/entry/dwight-ho...   

                                        short_description       date  \
0       She left her husband. He killed their children... 2018-05-26   
1                                Of course it has a song. 2018-05-26   
2       The actor and his longtime girlfriend Anna Ebe... 2018-05-26   
3       The actor gives Dems an ass-kicking for not fi... 2018-05-26   
4       The "Dietland" actress said using the bags is ... 2018-05-26   
...                                                   ...        ...   
200848  Verizon Wireless and AT&T are already promotin... 2012-01-28   
200849  Afterward, Azarenka, more effusive with the pr... 2012-01-28   
200850  Leading up to Super Bowl XLVI, the most talked... 2012-01-28   
200851  CORRECTION: An earlier version of this story i... 2012-01-28   
200852  The five-time all-star center tore into his te... 2012-01-28   

                                                     text  \
0       There Were 2 Mass Shootings In Texas Last Week...   
1       Will Smith Joins Diplo And Nicky Jam For The 2...   
2       Hugh Grant Marries For The First Time At Age 5...   
3       Jim Carrey Blasts 'Castrato' Adam Schiff And D...   
4       Julianna Margulies Uses Donald Trump Poop Bags...   
...                                                   ...   
200848  RIM CEO Thorsten Heins' 'Significant' Plans Fo...   
200849  Maria Sharapova Stunned By Victoria Azarenka I...   
200850  Giants Over Patriots, Jets Over Colts Among  M...   
200851  Aldon Smith Arrested: 49ers Linebacker Busted ...   
200852  Dwight Howard Rips Teammates After Magic Loss ...   

                                               clean_text  \
0       mass shoot texas last week leave husband kill ...   
1       smith join diplo nicky jam world cup official ...   
2       hugh grant marry first age actor longtime girl...   
3       jim carrey blast castrato adam schiff democrat...   
4       julianna margulies use donald trump poop bag p...   
...                                                   ...   
200848  rim ceo thorsten heins significant plan blackb...   
200849  maria sharapova stun victoria azarenka austral...   
2008

In [14]:
import gensim

word2vec_path = "GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=500000)

In [18]:
import numpy as np
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [19]:
df_news['avg_vec']=get_word2vec_embeddings(word2vec, df_news)

In [21]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(df_news,test_size=0.2, random_state=42)

In [22]:
from sklearn.linear_model import LogisticRegression
train_X=train['avg_vec'].values
train_y=train['category'].values

logreg = LogisticRegression(n_jobs=1,C=1e5)
logreg = logreg.fit(list(train_X),train_y)

C:\Users\sgmjin2\AppData\Local\conda\conda\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
labels=['ARTS', 'ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE',
       'COMEDY', 'CRIME', 'CULTURE & ARTS', 'DIVORCE', 'EDUCATION',
       'ENTERTAINMENT', 'ENVIRONMENT', 'FIFTY', 'FOOD & DRINK',
       'GOOD NEWS', 'GREEN', 'HEALTHY LIVING', 'HOME & LIVING', 'IMPACT',
       'LATINO VOICES', 'MEDIA', 'MONEY', 'PARENTING', 'PARENTS',
       'POLITICS', 'QUEER VOICES', 'RELIGION', 'SCIENCE', 'SPORTS',
       'STYLE', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'TRAVEL', 'WEDDINGS',
       'WEIRD NEWS', 'WELLNESS', 'WOMEN', 'WORLD NEWS', 'WORLDPOST']

In [25]:
from sklearn.metrics import classification_report, accuracy_score
y_pred=logreg.predict(list(test['avg_vec'].values))
print('accuracy %s'% accuracy_score(y_pred, test['category'].values))

accuracy 0.5453436558711509


In [26]:
print(classification_report(test['category'].values,y_pred, target_names=labels))

                precision    recall  f1-score   support

          ARTS       0.30      0.22      0.26       298
ARTS & CULTURE       0.28      0.12      0.17       283
  BLACK VOICES       0.40      0.28      0.33       882
      BUSINESS       0.44      0.42      0.43      1121
       COLLEGE       0.46      0.33      0.38       243
        COMEDY       0.36      0.22      0.27      1068
         CRIME       0.53      0.56      0.54       650
CULTURE & ARTS       0.51      0.18      0.26       222
       DIVORCE       0.69      0.66      0.67       696
     EDUCATION       0.38      0.27      0.31       188
 ENTERTAINMENT       0.49      0.62      0.55      3139
   ENVIRONMENT       0.44      0.15      0.22       275
         FIFTY       0.30      0.11      0.16       269
  FOOD & DRINK       0.60      0.72      0.65      1265
     GOOD NEWS       0.36      0.20      0.26       276
         GREEN       0.38      0.33      0.36       515
HEALTHY LIVING       0.38      0.12      0.18  

In [28]:
import pickle
model_name='avg_vec.mdl'
pickle.dump(logreg, open(model_name,'wb'))